In [2]:
import os
import sys

notebook_dir = os.path.abspath("C:\\Users\\katari.asish\\.vscode\\Semantic_Kernel\\statemang.ipynb")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)

# C:\Users\katari.asish\.vscode\Semantic_Kernel\statemang.ipynb
sys.path.append(grandparent_dir)

In [8]:
import asyncio
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.prompt_template import PromptTemplateConfig
from dotenv import dotenv_values

config = dotenv_values("Configuration.env")
kernel = Kernel()

In [14]:
azure_chat_service = AzureChatCompletion(
    service_id="ChatSessionRetain",
    deployment_name=config["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],  # Use the correct deployment name
    api_key=config["AZURE_OPENAI_API_KEY"],
    api_version=config["AZURE_OPENAI_API_VERSION"],
    endpoint=config["AZURE_OPENAI_ENDPOINT"]
)

In [15]:
azure_embedding_service = AzureTextEmbedding(
    service_id="ChatEmbedding",
    deployment_name=config["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"],  # Use the correct deployment name
    api_key=config["AZURE_OPENAI_API_KEY"],
    api_version=config["AZURE_OPENAI_API_VERSION"],
    endpoint=config["AZURE_OPENAI_ENDPOINT"]
)

In [16]:
kernel.add_service(azure_chat_service)
kernel.add_service(azure_embedding_service)

In [10]:
prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

In [17]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.connectors.ai.open_ai.services.azure_text_embedding import AzureTextEmbedding
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin
from semantic_kernel.kernel import Kernel
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore


memory = SemanticTextMemory(storage=VolatileMemoryStore(), embeddings_generator=azure_embedding_service)
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")



KernelPlugin(name='TextMemoryPlugin', description=None, functions={'recall': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='recall', plugin_name='TextMemoryPlugin', description='Recall a fact from the long term memory', parameters=[KernelParameterMetadata(name='ask', description='The information to retrieve', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'The information to retrieve'}), KernelParameterMetadata(name='collection', description='The collection to search for information.', default_value='generic', type_='str', is_required=False, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'The collection to search for information.'}), KernelParameterMetadata(name='relevance', description='The relevance score, from 0.0 to 1.0; 1.0 means perfect match', default_value=0.75, type_='float', is_required=False, type_object=<class 'float'>, schema_data={'type': 'number', 'descr

In [18]:
collection_id = "generic"

async def populate_memory(memory: SemanticTextMemory) -> None:
    # Add some documents to the semantic memory
    await memory.save_information(collection=collection_id, id="info1", text="Your budget for 2024 is $100,000")
    await memory.save_information(collection=collection_id, id="info2", text="Your savings from 2023 are $50,000")
    await memory.save_information(collection=collection_id, id="info3", text="Your investments are $80,000")

await populate_memory(memory)

async def search_memory_examples(memory: SemanticTextMemory) -> None:
    questions = [
        "What is my budget for 2024?",
        "What are my savings from 2023?",
        "What are my investments?",
    ]

    for question in questions:
        print(f"Question: {question}")
        result = await memory.search(collection_id, question)
        print(f"Answer: {result[0].text}\n")


await search_memory_examples(memory)

Question: What is my budget for 2024?
Answer: Your budget for 2024 is $100,000

Question: What are my savings from 2023?
Answer: Your savings from 2023 are $50,000

Question: What are my investments?
Answer: Your investments are $80,000

